<a href="https://colab.research.google.com/github/swaminaathakrishnan/MynewRepo/blob/master/Cool_route_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# CELL 1: SETUP & INSTALLATION
# ==========================================
!pip install osmnx simplekml geopandas shapely networkx requests

import osmnx as ox
import networkx as nx
import simplekml
import geopandas as gpd
import requests
import statistics
import math
from shapely.geometry import Point, LineString

print("✅ Environment Ready! Proceed to next cell.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 6.4 MB/s eta 0:00:00
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=064810d1a812a5f7d7b079eaa372d07ca584587c8dccdf3a9ab51c1c314c42e3
  Stored in directory: /root/.cache/pip/wheels/83/ee/f2/65cecfd948f1429ead035fd6d56bc6bd6574a636ddc4d65cbd
Successfully built simplekml
✅ Environment Ready! Proceed to next cell.


In [ ]:
# ==========================================
# CELL 2: CONFIGURATION
# ==========================================
# 1. Pilot Zone
PLACE_NAME = "Tampines, Singapore"

# 2. File Paths (Your Google Drive)
PCN_PATH = "/content/drive/MyDrive/YDCT/ParkConnectorLoop.geojson"
HAWKER_PATH = "/content/drive/MyDrive/YDCT/HawkerCentresGEOJSON.geojson"

# 3. Define Start & End Points (Lat, Long)
# Example: Tampines MRT -> Bedok Reservoir Park
START_POINT = (1.3533, 103.9452) # Tampines MRT
END_POINT = (1.3405, 103.9312)   # Bedok Reservoir Food Centre

print("✅ Configuration Set.")

✅ Configuration Set.


In [ ]:
# ==========================================
# CELL 3: THE "REAL DATA" ENGINE (AGGRESSIVE)
# ==========================================
def generate_cool_routes():
    print(f"⏳ Downloading road network for {PLACE_NAME}...")

    # 1. GET THE GRAPH
    # Dist increased to 3000m to allow wider detours
    G = ox.graph_from_point(START_POINT, dist=3000, network_type='bike')

    # 2. INJECT PARK CONNECTOR DATA
    print("⏳ Overlaying Government PCN Data...")
    try:
        pcn_data = gpd.read_file(PCN_PATH)
        if pcn_data.crs != "EPSG:4326": pcn_data = pcn_data.to_crs("EPSG:4326")
        pcn_union = pcn_data.geometry.unary_union
        print(f"   ✅ Loaded {len(pcn_data)} PCN segments.")
    except Exception as e:
        print(f"   ⚠️ PCN Error: {e}")
        pcn_union = None

    # 3. INJECT HAWKER DATA
    print("⏳ Loading Hawker Shelters...")
    shelters = []
    try:
        hawker_data = gpd.read_file(HAWKER_PATH)
        if hawker_data.crs != "EPSG:4326": hawker_data = hawker_data.to_crs("EPSG:4326")
        minx, miny, maxx, maxy = ox.utils_geo.bbox_from_point(START_POINT, dist=3000)
        hawker_data = hawker_data.cx[miny:maxy, minx:maxx]
        for _, row in hawker_data.iterrows():
            name = row.get('Name') or row.get('NAME') or "Cooling Shelter"
            shelters.append((name, row.geometry.y, row.geometry.x))
    except: pass

    # 4. MAP "COOL" SEGMENTS (The Aggressive Update)
    print("⏳ Analyzing Thermal Safety...")
    for u, v, k, data in G.edges(keys=True, data=True):
        # We need the geometry to check intersection accurately
        if 'geometry' in data:
            edge_geom = data['geometry']
        else:
            # Create straight line if geometry missing
            edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])])

        # COST LOGIC V2 (Aggressive)
        is_cool = False
        if pcn_union and (edge_geom.intersects(pcn_union) or edge_geom.distance(pcn_union) < 0.0001):
            is_cool = True

        # Force the split:
        if is_cool:
            # Super cheap cost. The algorithm LOVES this.
            data['cool_cost'] = data['length'] * 0.1
            data['type'] = "PCN"
        else:
            # Massive penalty. The algorithm HATES this.
            # 10x penalty means it will detour up to 10x distance just to stay cool.
            data['cool_cost'] = data['length'] * 10.0
            data['type'] = "Road"

    # 5. SOLVE PATHS
    print("⏳ Solving Routes...")
    orig = ox.distance.nearest_nodes(G, START_POINT[1], START_POINT[0])
    dest = ox.distance.nearest_nodes(G, END_POINT[1], END_POINT[0])

    try:
        route_fast = nx.shortest_path(G, orig, dest, weight='length')
        route_cool = nx.shortest_path(G, orig, dest, weight='cool_cost')

        # Check if they are identical (Debugging)
        if route_fast == route_cool:
            print("⚠️ Routes are identical. The destination might be too close or no PCN exists nearby.")
        else:
            print("✅ Routes Diverged! (Green path found a detour)")

        return G, route_fast, route_cool, shelters
    except nx.NetworkXNoPath:
        print("❌ Error: No path found.")
        return None, None, None, None

In [ ]:
# ==========================================
# CELL 4: WEATHER INTELLIGENCE (FINAL FIX)
# ==========================================
import requests
import math
import statistics
import simplekml
import json

def get_nearest_wbgt_station(target_lat, target_lon):
    print("⏳ Connecting to NEA Official WBGT Sensor Network...")
    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
    params = {"api": "wbgt"}

    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        readings = data['data']['records'][0]['item']['readings']

        nearest_station = "Unknown"
        min_dist = float('inf')
        wbgt_value = None

        # DEBUG: Confirming we see data
        # print(f"   🔍 RAW SAMPLE: {json.dumps(readings[0], indent=2)}")

        for r in readings:
            try:
                # 1. Extract Location (FIXED: Location is at root, not inside station)
                # Check directly in 'r' first
                if 'location' in r:
                    loc = r['location']
                elif 'station' in r and 'location' in r['station']:
                    # Fallback for old API structure
                    loc = r['station']['location']
                else:
                    continue

                s_lat = float(loc.get('latitude', 0))
                s_lon = float(loc.get('longitude', loc.get('longtitude', 0))) # Handle typo
                s_name = r.get('station', {}).get('name', 'Unknown')

                # 2. Extract Value
                raw_val = r.get('wbgt') or r.get('value')
                if raw_val is None or str(raw_val).strip() == "-": continue
                s_val = float(raw_val)

                # 3. Find Nearest
                dist = math.sqrt((target_lat - s_lat)**2 + (target_lon - s_lon)**2)
                if dist < min_dist:
                    min_dist = dist
                    nearest_station = s_name
                    wbgt_value = s_val
            except: continue

        if wbgt_value is None:
            return 32.0, "Simulation"

        print(f"   📍 Nearest Sensor: {nearest_station} (Dist: {min_dist*111:.2f} km)")
        print(f"   🌡️ Official WBGT: {wbgt_value}°C")
        return wbgt_value, nearest_station

    except Exception as e:
        print(f"   ⚠️ API Error: {e}. Defaulting to 32°C.")
        return 32.0, "Simulation"

def get_air_temp():
    try:
        url = "https://api-open.data.gov.sg/v2/real-time/api/air-temperature"
        data = requests.get(url, timeout=5).json()
        readings = [r['value'] for r in data['data']['readings'][0]['data']]
        return statistics.mean(readings)
    except:
        return 30.0

def export_to_kml(G, r_fast, r_cool, shelters):
    # 1. FETCH LIVE DATA
    wbgt, station_id = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    if wbgt is None: wbgt = 32.0
    air_temp = get_air_temp()

    kml = simplekml.Kml()

    # 2. RISK LOGIC (FIXED VARIABLE NAME)
    if wbgt < 29: risk="LOW"; color_code="🟢"
    elif wbgt < 31: risk="MODERATE"; color_code="🟡"
    elif wbgt < 33: risk="HIGH"; color_code="🟠"
    else: risk="EXTREME"; color_code="🔴"

    # 3. GEOMETRY HELPER
    def get_smooth_coords(graph, route):
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            edge_data = graph.get_edge_data(u, v)[0]
            if 'geometry' in edge_data:
                xs, ys = edge_data['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        return coords

    # 4. EXPORT ROUTES
    info_block = (
        f"<b>LIVE SENSOR DATA ({station_id})</b><br/>"
        f"WBGT: {wbgt}°C {color_code}<br/>"
        f"Heat Risk: {risk}<br/>"
        f"Air Temp: {air_temp:.1f}°C<br/>"
    )

    # Route A: Fast (Red)
    ls_fast = kml.newlinestring(name=f"Fastest Route (High Exposure)")
    ls_fast.coords = get_smooth_coords(G, r_fast)
    ls_fast.style.linestyle.color = simplekml.Color.red
    ls_fast.style.linestyle.width = 5
    ls_fast.description = info_block + "Path: Direct Road (High Exposure)<br/>Avg Radiant Load: High"

    # Route B: Cool (Green)
    ls_cool = kml.newlinestring(name="CoolRoute (Shade Optimized)")
    ls_cool.coords = get_smooth_coords(G, r_cool)
    ls_cool.style.linestyle.color = simplekml.Color.green
    ls_cool.style.linestyle.width = 5
    ls_cool.description = info_block + "Path: PCN/Park Network (Shaded)<br/>Avg Radiant Load: Low"

    # 5. SHELTERS
    for name, lat, lon in shelters:
        p = kml.newpoint(name=f"🧊 {name}", coords=[(lon, lat)])
        p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/snowflake_simple.png'

    filename = f"CoolRoute_Official_WBGT_{str(wbgt).replace('.','_')}.kml"
    kml.save(filename)
    print(f"\n🎉 SUCCESS! Generated official report: {filename}")

print("✅ Real-Time WBGT Engine (Final Fix) Ready.")

✅ Real-Time WBGT Engine (Final Fix) Ready.


In [ ]:
# ==========================================
# CELL 4.5 V9: PAGINATION-ENABLED AI ENGINE
# ==========================================
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import numpy as np
import requests
import pickle
import os
import time

CACHE_FILE = "coolride_weather_memory.pkl"

def get_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "rb") as f:
                return pickle.load(f)
        except: return {}
    return {}

def save_cache(data):
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(data, f)

def fetch_historical_data(station_name, days_back=3):
    # 1. CHECK CACHE
    cache = get_cache()
    today_str = datetime.now().strftime("%Y-%m-%d")
    cache_key = f"{station_name}_{today_str}"

    if cache_key in cache:
        count = len(cache[cache_key]['values'])
        print(f"   ⚡ Cache Hit! Loaded {count} historical points.")
        return cache[cache_key]['timestamps'], cache[cache_key]['values']

    # 2. FETCH FROM API (WITH PAGINATION)
    print(f"   📡 Cache Miss. Starting {days_back}-Day Historical Analysis...")
    all_timestamps = []
    all_values = []

    for i in range(days_back + 1):
        target_date = (datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d")
        url = "https://api-open.data.gov.sg/v2/real-time/api/weather"

        # Start with no token
        params = {"api": "wbgt", "date": target_date}

        print(f"      👉 Querying {target_date}...", end=" ")

        day_records = []
        page_count = 0

        while True:
            try:
                # Timeout 5s per page
                resp = requests.get(url, params=params, timeout=5)

                if resp.status_code != 200: break

                data = resp.json()
                if 'data' not in data: break

                # Add records from this page
                new_records = data['data'].get('records', [])
                day_records.extend(new_records)
                page_count += 1

                # STOPPING CONDITION: Check if pagination token exists
                token = data['data'].get('paginationToken')
                if token:
                    params['paginationToken'] = token # Set up next page
                    # Safety break to prevent infinite loops during demo (max 50 pages)
                    if page_count > 50: break
                else:
                    break # No more pages

            except: break

        print(f"✅ Downloaded {len(day_records)} raw logs (in {page_count} pages).", end=" ")

        # EXTRACT RELEVANT DATA
        day_points = 0
        for rec in day_records:
            try:
                dt = datetime.fromisoformat(rec['datetime'])
                for r in rec['item']['readings']:
                    name = r.get('station', {}).get('name')
                    if name == station_name:
                        val = float(r.get('wbgt') or r.get('value'))

                        # Time Alignment
                        minutes = dt.hour * 60 + dt.minute
                        now_minutes = datetime.now().hour * 60 + datetime.now().minute

                        # +/- 4 Hours Window
                        if abs(minutes - now_minutes) < 240:
                            all_timestamps.append(minutes)
                            all_values.append(val)
                            day_points += 1
            except: continue

        print(f"-> Extracted {day_points} relevant.")

    print(f"   🔍 Total Training Data: {len(all_values)} points.")

    # 3. SAVE TO CACHE
    if len(all_values) > 10:
        cache[cache_key] = {'timestamps': all_timestamps, 'values': all_values}
        save_cache(cache)
        print("   💾 History saved to local memory.")
    else:
        print("   ⚠️ Data sparse. Not caching.")

    return all_timestamps, all_values

def predict_short_term_trend(station_name, current_wbgt):
    print(f"🧠 Training AI Model for: {station_name}...")

    timestamps, values = fetch_historical_data(station_name, days_back=3)

    if len(values) < 5:
        print(f"   ⚠️ Fallback: Persistence model.")
        return current_wbgt, "Stable ➖", "Low (Data Scarcity)"

    # TRAIN
    X = np.array(timestamps).reshape(-1, 1)
    y = np.array(values)
    model = LinearRegression()
    model.fit(X, y)

    # PREDICT
    now = datetime.now()
    cur_min = now.hour * 60 + now.minute
    raw_pred = model.predict([[cur_min + 15]])[0]

    # CLAMP
    MAX_CHANGE = 0.5
    delta = raw_pred - current_wbgt

    if abs(delta) > MAX_CHANGE:
        final_pred = current_wbgt + (0.5 if delta > 0 else -0.5)
        clamp_note = "(Clamped)"
    else:
        final_pred = raw_pred
        clamp_note = ""

    # STATS
    y_pred_hist = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred_hist))
    conf_pct = max(0, 100 - (rmse * 20))

    if final_pred > current_wbgt + 0.1: trend = "Rising 📈"
    elif final_pred < current_wbgt - 0.1: trend = "Falling 📉"
    else: trend = "Stable ➖"

    confidence_str = f"{int(conf_pct)}% {clamp_note}"

    print(f"   📊 AI Model: Linear Regression (n={len(values)})")
    print(f"   🔮 Forecast (+15m): {final_pred:.2f}°C ({trend})")
    print(f"   🛡️ Confidence: {confidence_str}")

    return final_pred, trend, confidence_str

print("✅ Pagination-Enabled AI Engine Ready.")

✅ Pagination-Enabled AI Engine Ready.


In [ ]:
# ==========================================
# DEFENSE IN DEPTH: GOVERNMENT OVERRIDE
# ==========================================
# Toggle this manually based on NEA News/SMS
NEA_HEATWAVE_ALERT = False # Set to True during demo to show "Fail-Safe"

# ==========================================
# CELL 5: EXECUTE PROTOTYPE (ACTIVE SAFETY AI)
# ==========================================
print("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")

# 1. Generate Routes
graph, r1, r2, shelters = generate_cool_routes()

if graph and r1 and r2:
    # 2. Get Real-Time Data
    current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    # 3. Run AI Prediction
    # We add 'confidence' to the unpacking as per previous update
    if current_wbgt:
        pred_wbgt, trend, confidence = predict_short_term_trend(station_name, current_wbgt)
    else:
        current_wbgt = 32.0; pred_wbgt = 32.0; trend = "Simulated"; confidence = "N/A"

    # --- CRITICAL UPDATE: THE "SO WHAT?" LOGIC ---
    # We route based on the WORST case scenario (Current vs Future)
    effective_wbgt = max(current_wbgt, pred_wbgt)

    # 2. The Override (During Govt declared emergency time)
    if NEA_HEATWAVE_ALERT:
        print("\n🚨 GOVERNMENT ADVISORY ACTIVE: FORCING MAX SAFETY MODE.")
        print("   (Ignoring sensor readings due to national alert)")
        effective_wbgt = 35.0  # Force it to EXTREME RISK
        confidence = "MAX (Gov Advisory)"

    print(f"\n⚖️ SAFETY DECISION ENGINE:")
    print(f"   Current: {current_wbgt}°C | Forecast: {pred_wbgt:.1f}°C")
    print(f"   👉 System Optimization Target: {effective_wbgt:.1f}°C")

    # 4. Export KML
    print("\n💾 Exporting Intelligent KML...")
    kml = simplekml.Kml()

   # Risk Logic now uses EFFECTIVE WBGT
    if effective_wbgt < 29: risk="LOW"; color="🟢"
    elif effective_wbgt < 31: risk="MODERATE"; color="🟡"
    elif effective_wbgt < 33: risk="HIGH"; color="🟠"
    else: risk="EXTREME"; color="🔴"

    # Info Block
    info_block = (
        f"<b>INTELLIGENT SENSOR DATA ({station_name})</b><br/>"
        f"Now: {current_wbgt}°C<br/>"
        f"<b>Forecast (+15m): {pred_wbgt:.1f}°C {trend}</b><br/>"
        f"AI Confidence: {confidence}<br/>"
        f"Risk Level: {risk} {color}<br/>"
    )

    # Helper: Smooth Coords
    def get_smooth_coords(graph, route):
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            edge_data = graph.get_edge_data(u, v)[0]
            if 'geometry' in edge_data:
                xs, ys = edge_data['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        return coords

    # Route A (Red)
    ls_fast = kml.newlinestring(name=f"Fastest Route ({risk})")
    ls_fast.coords = get_smooth_coords(graph, r1)
    ls_fast.style.linestyle.color = simplekml.Color.red
    ls_fast.style.linestyle.width = 5
    ls_fast.description = info_block + "Path: Direct Road (High Exposure)"

    # Route B (Green)
    ls_cool = kml.newlinestring(name=f"CoolRoute (AI Safety Buffer)")
    ls_cool.coords = get_smooth_coords(graph, r2)
    ls_cool.style.linestyle.color = simplekml.Color.green
    ls_cool.style.linestyle.width = 5
    ls_cool.description = info_block + "Path: PCN/Park Network (Shaded)"

    # Shelters
    for name, lat, lon in shelters:
        p = kml.newpoint(name=f"🧊 {name}", coords=[(lon, lat)])
        p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/snowflake_simple.png'

    filename = f"CoolRoute_AI_Safety_{str(effective_wbgt).replace('.','_')}.kml"
    kml.save(filename)
    print(f"\n🎉 SUCCESS! Generated AI-Active report: {filename}")
else:
    print("❌ Route Generation Failed.")

🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...
⏳ Downloading road network for Tampines, Singapore...
⏳ Overlaying Government PCN Data...
   ✅ Loaded 883 PCN segments.
⏳ Loading Hawker Shelters...
⏳ Analyzing Thermal Safety...


/tmp/ipython-input-358972399.py:16: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  pcn_union = pcn_data.geometry.unary_union


⏳ Solving Routes...
✅ Routes Diverged! (Green path found a detour)
⏳ Connecting to NEA Official WBGT Sensor Network...
   📍 Nearest Sensor: Bedok North Street 2 (Dist: 3.12 km)
   🌡️ Official WBGT: 25.7°C
🧠 Training AI Model for: Bedok North Street 2...
   ⚡ Cache Hit! Loaded 128 historical points.
   📊 AI Model: Linear Regression (n=128)
   🔮 Forecast (+15m): 25.20°C (Falling 📉)
   🛡️ Confidence: 75% (Clamped)

⚖️ SAFETY DECISION ENGINE:
   Current: 25.7°C | Forecast: 25.2°C
   👉 System Optimization Target: 25.7°C

💾 Exporting Intelligent KML...

🎉 SUCCESS! Generated AI-Active report: CoolRoute_AI_Safety_25_7.kml


In [ ]:
import os
if os.path.exists("coolride_weather_memory.pkl"):
    os.remove("coolride_weather_memory.pkl")
    print("🗑️ Cache Deleted! The AI is now forced to re-learn.")
else:
    print("✅ Cache already clear.")

🗑️ Cache Deleted! The AI is now forced to re-learn.
